# MONGY: Training `PatchTSMixer` on Financial Candlestick Data
## Direct forecasting example

This notebooke demonstrates the usage of a `PatchTSMixer` model for a multivariate time series forecasting task. This notebook has a dependecy on HuggingFace [transformers](https://github.com/huggingface/transformers) repo. For details related to model architecture, refer to the [TSMixer paper](https://arxiv.org/abs/2306.09364).

In [1]:
# Standard
import os
import random

# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
)
import numpy as np
import pandas as pd
import torch

# First Party
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor

In [2]:
# Set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [3]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="torch.utils.data.dataloader")

## Load and prepare datasets

In the next cell, please adjust the following parameters to suit your application:
- `dataset_path`: path to local .csv file, or web address to a csv file for the data of interest. Data is loaded with pandas, so anything supported by
`pd.read_csv` is supported: (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
- `timestamp_column`: column name containing timestamp information, use None if there is no such column
- `id_columns`: List of column names specifying the IDs of different time series. If no ID column exists, use []
- `forecast_columns`: List of columns to be modeled
- `context_length`: The amount of historical data used as input to the model. Windows of the input time series data with length equal to
context_length will be extracted from the input dataframe. In the case of a multi-time series dataset, the context windows will be created
so that they are contained within a single time series (i.e., a single ID).
- `forecast_horizon`: Number of time stamps to forecast in future.
- `train_start_index`, `train_end_index`: the start and end indices in the loaded data which delineate the training data.
- `valid_start_index`, `valid_end_index`: the start and end indices in the loaded data which delineate the validation data.
- `test_start_index`, `test_end_index`: the start and end indices in the loaded data which delineate the test data.
- `patch_length`: The patch length for the `PatchTSMixer` model. Recommended to have a value so that `context_length` is divisible by it.
- `num_workers`: Number of dataloder workers in pytorch dataloader.
- `batch_size`: Batch size. 
The data is first loaded into a Pandas dataframe and split into training, validation, and test parts. Then the pandas dataframes are converted
to the appropriate torch dataset needed for training.

In [4]:
# Load the Dataset from the CSV file
DATA_DIR = "/home/ubuntu/verb-workspace/data"

NUM_MINUTES = 5

TRAIN_DATASET = f"{DATA_DIR}/{NUM_MINUTES}min-candles-train-MORNING.csv"
VALID_DATASET = f"{DATA_DIR}/{NUM_MINUTES}min-candles-valid-MORNING.csv"
TEST_DATASET = f"{DATA_DIR}/{NUM_MINUTES}min-candles-test-MORNING.csv"

timestamp_col = 't'

train_data = pd.read_csv(
    TRAIN_DATASET,
    parse_dates=[timestamp_col]
)

valid_data = pd.read_csv(
    VALID_DATASET,
    parse_dates=[timestamp_col]
)

test_data = pd.read_csv(
    TEST_DATASET,
    parse_dates=[timestamp_col]
)


In [7]:
# We want to setup our context, horizon, and patch size based on our task. We want to use
# 4 hours of lookback to start, in order to predict the next 5 minutes of candles. Regarding
# patch length, we know that we will want a larger patch size, so we will start with 64 as
# a base case assumption

context_length = (60 * 4) // NUM_MINUTES  # This will give us 4 hours of lookback (4 hours * 60 min per hour) / 3 min per candle
forecast_horizon = 3 # This will give us 3 minutes of predictions

In [8]:
# Check for NaN values
assert sum(train_data.isna().sum().to_list()) == 0
assert sum(valid_data.isna().sum().to_list()) == 0
assert sum(test_data.isna().sum().to_list()) == 0

In [9]:
train_data

,ticker,date_string,t,targ_o,targ_h,targ_l,targ_c,targ_v,targ_vwap,targ_red,targ_green,cont_market_open,cont_market_extended
0,AAPL,2023-01-03,2023-01-03 05:30:00-05:00,130.800,130.80,130.8000,130.8000,235.0,130.800000,0,0,0.0,1.0
1,AAPL,2023-01-03,2023-01-03 05:35:00-05:00,130.800,130.80,130.8000,130.8000,0.0,0.000000,0,0,0.0,1.0
2,AAPL,2023-01-03,2023-01-03 05:40:00-05:00,130.800,131.02,130.8000,131.0000,4873.0,130.972664,0,1,0.0,1.0
3,AAPL,2023-01-03,2023-01-03 05:45:00-05:00,130.990,131.05,130.9900,131.0400,1535.0,131.041166,0,1,0.0,1.0
4,AAPL,2023-01-03,2023-01-03 05:50:00-05:00,131.020,131.04,131.0000,131.0000,200.0,131.000000,1,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99691,V,2023-11-17,2023-11-17 10:40:00-05:00,249.460,249.55,249.2600,249.4600,56185.5,249.443114,0,0,1.0,0.0
99692,V,2023-11-17,2023-11-17 10:45:00-05:00,249.460,249.58,249.4157,249.5150,60720.0,249.508379,0,1,1.0,0.0
99693,V,2023-11-17,2023-11-17 10:50:00-05:00,249.515,249.54,249.2800,249.4735,50726.0,249.463223,1,0,1.0,0.0
99694,V,2023-11-17,2023-11-17 10:55:00-05:00,249.470,249.47,249.1700,249.1700,52213.0,249.310552,1,0,1.0,0.0


In [11]:

id_columns = ['ticker', 'date_string']
forecast_columns = ['targ_o', 'targ_c', 'targ_h', 'targ_l', 'targ_v', 'targ_vwap', 'targ_red', 'targ_green']
control_columns = ['cont_market_open', 'cont_market_extended']

train_tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_col,
    id_columns=id_columns,
    target_columns=forecast_columns,
    control_columns=control_columns,
    scaling=True,
)
train_tsp.train(train_data)
print("Done Train")

valid_tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_col,
    id_columns=id_columns,
    target_columns=forecast_columns,
    control_columns=control_columns,
    scaling=True,
)
valid_tsp.train(valid_data)
print("Done Valid")

test_tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_col,
    id_columns=id_columns,
    target_columns=forecast_columns,
    control_columns=control_columns,
    scaling=True,
)
test_tsp.train(test_data)
print("Done Test")


Done Train
Done Valid
Done Test


In [12]:
train_dataset = ForecastDFDataset(
    train_tsp.preprocess(train_data),
    id_columns=id_columns,
    target_columns=forecast_columns,
    control_columns=control_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
print("Done Train")

valid_dataset = ForecastDFDataset(
    valid_tsp.preprocess(valid_data),
    id_columns=id_columns,
    target_columns=forecast_columns,
    control_columns=control_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
print("Done Valid")

test_dataset = ForecastDFDataset(
    test_tsp.preprocess(test_data),
    id_columns=id_columns,
    target_columns=forecast_columns,
    control_columns=control_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
print("Done Test")



Done Train
Done Valid
Done Test


In [13]:
from typing import Tuple
# Indices for accessing the OHLC values in the tensors
I_OPEN = 0
I_CLOSE = 1
I_HIGH = 2
I_LOW = 3
I_VOLUME = 4
I_VWAP = 5
I_RED = 6
I_GREEN = 7
    

def theta_body(y_pred: torch.Tensor, y_obs: torch.Tensor) -> torch.Tensor:
    # Create the series of closes
    real_candle_closes = y_obs[..., I_CLOSE]
    forecasted_candle_closes = y_pred[..., I_CLOSE]

    # Get the series of opens
    real_candle_opens = y_obs[..., I_OPEN]
    forecasted_candle_opens = y_pred[..., I_OPEN]

    # Get the series of candle bodies
    real_bodies = real_candle_closes - real_candle_opens
    forecasted_bodies = forecasted_candle_closes - forecasted_candle_opens

    # Get the error of each body
    error = real_bodies - forecasted_bodies

    sq_error = torch.mean(torch.square(error))
    abs_error = torch.mean(torch.abs(error))
    return sq_error, abs_error

def theta_single_pnl(x: torch.Tensor, y_pred: torch.Tensor, y_obs: torch.Tensor) -> torch.Tensor:
    # Get the close of the last real candle
    last_candle_close = x[..., -1, I_CLOSE]
    real_last_candle_close = y_obs[..., -1, I_CLOSE]
    real_forecasted_candle_close = y_pred[..., -1, I_CLOSE]

    # Compute if the position should be long or short, based on the real values
    is_long = real_last_candle_close > last_candle_close

    # Compute P/L of long position
    pnl_long_real = real_last_candle_close - last_candle_close
    pnl_long_forecasted = real_forecasted_candle_close - last_candle_close
    # Compute P/L of short position
    pnl_short_real = last_candle_close - real_last_candle_close
    pnl_short_forecasted = last_candle_close - real_forecasted_candle_close

    # Compute the P/L of the real candles vs forecasted candles
    pnl_real = torch.where(is_long, pnl_long_real, pnl_short_real)
    pnl_forecasted = torch.where(is_long, pnl_long_forecasted, pnl_short_forecasted)

    # Compute the P/L residuals. Here if the model's gain is larger than 
    # the actual gain, the pnl_error will be positive
    raw_pnl_error = pnl_forecasted - pnl_real
    
    # Where the model gives a greater prediction, we want to scale the model's
    # P/L error by a log func, to place a greater penalty on losing money over 
    # predicting too large a gain
    error = torch.where(
        raw_pnl_error > 0,
        torch.log(1 + raw_pnl_error) / 2,
        raw_pnl_error
    )

    sq_error = torch.square(error)
    abs_error = torch.abs(error)
    return sq_error, abs_error

def base_error(y_pred: torch.Tensor, y_obs: torch.Tensor) -> Tuple[torch.Tensor]:
    # Place a mask over y_obs, to ensure it is the same size as y_pred
    y_obs = y_obs[..., :y_pred.shape[-1]]
    
    error = y_obs - y_pred

    raw_sq_error = torch.mean(torch.square(error), dim=-1)
    raw_ae_error = torch.mean((torch.abs(error)), dim=-1)

    sq_error = torch.mean(raw_sq_error)
    ae_error = torch.mean(raw_ae_error)
    
    return sq_error, ae_error


def custom_loss(x: torch.Tensor, y_pred: torch.Tensor, y_obs: torch.Tensor) -> torch.Tensor:
    # Compute PNL rediual for each candle
    mse, mae = base_error(y_pred, y_obs)
    pnl_se, pnl_ae = theta_single_pnl(x, y_pred, y_obs)
    body_se, body_ae = theta_body(y_pred, y_obs)

    custom_mse = torch.mean(mse + pnl_se)
    custom_mae = torch.mean(mae + pnl_ae)

    return (custom_mse + custom_mae) / 2

In [14]:
# Indices for accessing the OHLC values in the tensors

from typing import Optional

from transformers.models.patchtsmixer.modeling_patchtsmixer import PatchTSMixerForPredictionOutput

class MongyModel(PatchTSMixerForPrediction):

    def forward(
        self,
        past_values: torch.Tensor,
        observed_mask: Optional[torch.Tensor] = None,
        future_values: Optional[torch.Tensor] = None,
        output_hidden_states: Optional[bool] = False,
        return_loss: bool = True,
        return_dict: Optional[bool] = None,
    ) -> PatchTSMixerForPredictionOutput:
        r"""
        observed_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length, num_input_channels)`, *optional*):
            Boolean mask to indicate which `past_values` were observed and which were missing. Mask values selected
            in `[0, 1]`:
                - 1 for values that are **observed**,
                - 0 for values that are **missing** (i.e. NaNs that were replaced by zeros).
        future_values (`torch.FloatTensor` of shape `(batch_size, target_len, num_input_channels)` for forecasting,:
            `(batch_size, num_targets)` for regression, or `(batch_size,)` for classification, *optional*): Target
            values of the time series, that serve as labels for the model. The `future_values` is what the
            Transformer needs during training to learn to output, given the `past_values`. Note that, this is NOT
            required for a pretraining task.

            For a forecasting task, the shape is be `(batch_size, target_len, num_input_channels)`. Even if we want
            to forecast only specific channels by setting the indices in `prediction_channel_indices` parameter,
            pass the target data with all channels, as channel Filtering for both prediction and target will be
            manually applied before the loss computation.
        return_loss (`bool`,  *optional*):
            Whether to return the loss in the `forward` call.

        Returns:

        """
        if self.loss == "mse":
            # loss = torch.nn.MSELoss(reduction="mean")
            loss = custom_loss
        elif self.loss == "nll":
            loss = nll
        else:
            raise ValueError("Invalid loss function: Allowed values: mse and nll")

        return_dict = return_dict if return_dict is not None else self.use_return_dict

        # past_values: tensor [batch_size x context_length x num_input_channels]
        model_output = self.model(
            past_values,
            observed_mask=observed_mask,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )  # model_output: [batch_size x nvars x num_patch x d_model]

        
        if isinstance(model_output, tuple):
            model_output = PatchTSMixerModelOutput(*model_output)

        # tensor [batch_size x prediction_length x num_input_channels]
        y_hat = self.head(model_output.last_hidden_state)

        # Snap the candles to the correct opening positions, before computing the loss
        # This is the "training wheels" for the head. By helping the model with the portion
        # of it's task that we can help with, we severly limit the task that is posed to the
        # model
        last_context_close = past_values[..., -1, I_CLOSE]
        first_candle_open = y_hat[..., 0, I_OPEN]
        first_candle_delta = last_context_close - first_candle_open
        first_candle_delta = first_candle_delta.unsqueeze(-1).unsqueeze(-1)
        y_hat[..., 0:4] = y_hat[..., 0:4] + first_candle_delta


        first_candle_close = y_hat[..., 0, I_CLOSE]
        second_candle_open = y_hat[..., 1, I_OPEN]
        second_candle_delta = first_candle_close - second_candle_open
        second_candle_delta = second_candle_delta.unsqueeze(-1).unsqueeze(-1)
        y_hat[..., -2:, 0:4] = y_hat[..., -2:, 0:4] + second_candle_delta

        second_candle_close = y_hat[..., 1, I_CLOSE]
        third_candle_open = y_hat[..., 2, I_OPEN]
        third_candle_delta = second_candle_close - third_candle_open
        third_candle_delta = third_candle_delta.unsqueeze(-1)
        y_hat[..., -1, 0:4] = y_hat[..., -1, 0:4] + third_candle_delta


        loss_val = None
        if self.prediction_channel_indices is not None:
            if self.distribution_output:
                distribution = self.distribution_output.distribution(
                    y_hat,
                    loc=model_output.loc[..., self.prediction_channel_indices],
                    scale=model_output.scale[..., self.prediction_channel_indices],
                )
                if future_values is not None and return_loss is True:
                    loss_val = loss(
                        distribution,
                        future_values[..., self.prediction_channel_indices],
                    )
                    # take average of the loss
                    loss_val = weighted_average(loss_val)
            else:
                y_hat = (
                    y_hat * model_output.scale[..., self.prediction_channel_indices]
                    + model_output.loc[..., self.prediction_channel_indices]
                )
                if future_values is not None and return_loss is True:
                    loss_val = loss(past_values, y_hat, future_values[..., self.prediction_channel_indices])
        else:
            if self.distribution_output:
                distribution = self.distribution_output.distribution(
                    y_hat, loc=model_output.loc, scale=model_output.scale
                )
                if future_values is not None and return_loss is True:
                    loss_val = loss(distribution, future_values)
                    loss_val = weighted_average(loss_val)
            else:
                y_hat = y_hat * model_output.scale + model_output.loc
                if future_values is not None and return_loss is True:
                    loss_val = loss(y_hat, future_values)

        if self.prediction_channel_indices is not None:
            loc = model_output.loc[..., self.prediction_channel_indices]
            scale = model_output.scale[..., self.prediction_channel_indices]
        else:
            loc = model_output.loc
            scale = model_output.scale

        if not return_dict:
            return tuple(
                v
                for v in [
                    loss_val,
                    y_hat,
                    model_output.last_hidden_state,
                    model_output.hidden_states,
                    loc,
                    scale,
                ]
            )

        return PatchTSMixerForPredictionOutput(
            loss=loss_val,
            prediction_outputs=y_hat,  # tensor [batch_size x prediction_length x num_input_channels]
            last_hidden_state=model_output.last_hidden_state,  # x: [batch_size x nvars x num_patch x d_model]
            hidden_states=model_output.hidden_states,
            loc=loc,
            scale=scale,
        )

## Training `PatchTSMixer` From Scratch

Adjust the following model parameters according to need.
- `d_model` (`int`, *optional*, defaults to 8):
    Hidden dimension of the model. Recommended to set it as a multiple of patch_length (i.e. 2-8X of
    patch_len). Larger value indicates more complex model.
- `expansion_factor` (`int`, *optional*, defaults to 2):
    Expansion factor to use inside MLP. Recommended range is 2-5. Larger value indicates more complex model.
- `num_layers` (`int`, *optional*, defaults to 3):
    Number of layers to use. Recommended range is 3-15. Larger value indicates more complex model.
- `mode`: (`str`, either to 'common_channel' or `mix_channel`)

In [15]:
patch_length = 16
stride_length = 1

MODEL_CHKPT = "./saved_checkpoints/morning_3min/output/checkpoint-32000"

prediction_channel_indicies = train_tsp.prediction_channel_indices
num_input_channels = train_tsp.num_input_channels

config = PatchTSMixerConfig(
    # Dataset Kwargs
    context_length=context_length,
    prediction_length=forecast_horizon,
    prediction_channel_indices=prediction_channel_indicies,
    patch_length=patch_length,
    num_input_channels=num_input_channels,
    patch_stride=stride_length,

    # Model Kwargs
    d_model=6 * patch_length,
    num_layers=4,
    expansion_factor=3,
    dropout=0.5,
    head_dropout=0.7,
    mode="mix_channel",
    scaling=None,
)


if not MODEL_CHKPT:
    model = MongyModel(config=config)
else:
    model = MongyModel.from_pretrained(MODEL_CHKPT, config=config, ignore_mismatched_sizes=True)


Some weights of MongyModel were not initialized from the model checkpoint at ./saved_checkpoints/morning_3min/output/checkpoint-32000 and are newly initialized because the shapes did not match:
- head.base_forecast_block.weight: found shape torch.Size([3, 6240]) in the checkpoint and torch.Size([3, 3168]) in the model instantiated
- model.encoder.mlp_mixer_encoder.mixers.0.patch_mixer.gating_block.attn_layer.bias: found shape torch.Size([65]) in the checkpoint and torch.Size([33]) in the model instantiated
- model.encoder.mlp_mixer_encoder.mixers.0.patch_mixer.gating_block.attn_layer.weight: found shape torch.Size([65, 65]) in the checkpoint and torch.Size([33, 33]) in the model instantiated
- model.encoder.mlp_mixer_encoder.mixers.0.patch_mixer.mlp.fc1.bias: found shape torch.Size([195]) in the checkpoint and torch.Size([99]) in the model instantiated
- model.encoder.mlp_mixer_encoder.mixers.0.patch_mixer.mlp.fc1.weight: found shape torch.Size([195, 65]) in the checkpoint and torch.Si

# Training Run Summaries

**Run 1**: (N/A)
This run used the full year of data, and was used as a baseline to establish that the `mix_channel` mode is more effective for our task. Additionally, all subsequent runs have been updated, to instead use only the first three months of data as training data. Thus, while the loss for this run is lower, it is not indicaitve of the paramters being a better fit, just a result of having a larger dataset.

**Run 2** (0.108476):
This run was the first in which only the first two months of data was used as a training set. March was then split in half to form the validation and test sets. Additionally, the context window was expanded, to include the last four hours of data. While this wasn't explicitly compared against a shorter context window with the same dataset, the results of the paper provide an incredibly strong suggestions towards this approach yielding more effective performance.

**Run 3** (0.108230):
This run included involved increasing the `num_layers` argument from 3 to 5. This adds additional layers to the model, giving it more of an ability to percieve complex patterns in the financial data. This results in a larger model, but hopefully, will allow the model to better understand the nuances of the highly complex financial data it is being trained on.

**Run 4**: (0.107247)
This run included further incrementing the `num_layers` argument from 5 to 10. This adds additional further layers to capture more of the complex patterns in the financial dataset. 

_NOTE_: The `num_layers` does not seem to provide additional aid in this trainin task, with the side-effect of signifitcanlty increasing the inference time. As a result, we are making the decision to keep `num_layers = 3`.

---

**Run 5**: (0.108397)
The `num_layers` argument has been reset to a value of 3, which returns our baseline back to _Run 2_. The `expansion_factor` has been increased from 3 to 4. This yeilded a slight decrease in validation loss, so potentially worth running a second experiment, but likely best to test patching instead.

**Run 6** ()

In [16]:
# Compute the run number
run_num = "morning_1_5min"
save_dir = f"./model_checkpoints/run_{run_num}"
torch.cuda.empty_cache()

# Check if save_dir exists
assert not os.path.exists(save_dir), "Please update the run_num to avoid overwriting checkpoints!"

num_workers = 10  # p3.2xlarge instance has 12 vCPUs

gradient_accumulation_steps = 1 # Number of batches between each backward pass
batch_size = 64 # Size of each batches sent to GPU
eval_batch_size = 256
num_steps = 1000

# Calculations
# =======================
# effective_batch_size = batch_size * grad_accumulation_steps = 64 * 1 = 64
# examples_per_evaluation = num_steps * effective_batch_size = 64 * 2,500 = 160,0000

train_args = TrainingArguments(
    output_dir=f"{save_dir}/output/",
    overwrite_output_dir=True,
    learning_rate=0.000001,
    weight_decay=0.0000005,
    num_train_epochs=100,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=num_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    eval_accumulation_steps=250,
    dataloader_num_workers=num_workers,
    report_to="tensorboard",
    save_strategy="steps",
    save_steps=num_steps,
    logging_strategy="steps",
    logging_steps=num_steps,
    save_total_limit=3,
    logging_dir=f"{save_dir}/logs/",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"], # The names of the "ground truth" values to compare predictions against
)

# Create a new early stopping callback with faster convergence properties
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
)

print("Doing forecasting training on Dataset")
trainer.train()

Doing forecasting training on Dataset


Step,Training Loss,Validation Loss
1000,3.475900,1.995783
2000,3.433600,1.949412
3000,3.347300,1.910144
4000,3.298800,1.876469
5000,3.227600,1.847231
6000,3.211700,1.821607
7000,3.147200,1.799353
8000,3.112900,1.779075
9000,3.056400,1.761258
10000,3.016900,1.745599


TrainOutput(global_step=39000, training_loss=2.882934845753205, metrics={'train_runtime': 2584.2463, 'train_samples_per_second': 978.854, 'train_steps_per_second': 15.324, 'total_flos': 2206743758346240.0, 'train_loss': 2.882934845753205, 'epoch': 98.48484848484848})

In [15]:
trainer.evaluate(test_dataset)

{'eval_loss': 1.0075856447219849,
 'eval_runtime': 29.835,
 'eval_samples_per_second': 557.834,
 'eval_steps_per_second': 2.212,
 'epoch': 21.73913043478261}